# Calculation of chisquare of the GMA database

This notebook shows how to do a fit with the `gmapy` package and determine the chisquare value of the database.


## Load necessary libraries

In [1]:
import sys
sys.path.append('../..')  # so that gmapy can be found
import numpy as np
from scipy.sparse.linalg import spsolve
from gmapy.legacy.legacy_gmap import run_gmap
from gmapy.mappings.compound_map import CompoundMap
dbfile = '../../tests/testdata/data-2017-07-26.gma'

## Perform the fit

In [2]:
%%capture
res = run_gmap(dbfile, dbtype='legacy', num_iter=3, fix_ppp_bug=True, fix_sacs_jacobian=True, legacy_output=False)

## Calculate the chisquare by df value

Prepare the quantities needed to compute the chisquare value.

In [3]:
compmap = CompoundMap()
datatable = res['table'].copy()
postvals = datatable['POST'].to_numpy()
preds = compmap.propagate(postvals, datatable)
expsel = datatable.NODE.str.match('exp_')
expidcs = np.ravel(np.array(np.where(expsel)))
expvals = datatable.loc[expidcs, 'DATA'].to_numpy()
preds = preds[expidcs]
d = expvals - preds
expcov = res['priorcov'][np.ix_(expidcs, expidcs)]

Evaluate the chisquare by df value

In [4]:
chisqr_df = float(d.T @ spsolve(expcov, d) / len(expvals))
print(chisqr_df)

0.7961857222863135


Remove the dummy datapoints as they are always set to equal the prediction and recompute the chisquare by df value

In [5]:
sel = np.logical_not(datatable.loc[expidcs,'NODE'].str.match('exp_90[0-9]'))
chisqr_df2 = float(d[sel].T @ spsolve(expcov[np.ix_(sel,sel)], d[sel]) / len(expvals[sel]))
print(chisqr_df2)

0.8922323463074535
